In [10]:
import os
import sys
import git
data_path = '/home/cip/ce/ix05ogym/Majid/LLM/llm_made/data/'
url_list = ['https://github.com/yannickrud/made-project.git',
            'https://github.com/tosiful/made-tim.git']
for i,url in enumerate(url_list):
    
    git.Repo.clone_from(url,data_path+f'{i}/')
    



GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://github.com/yannickrud/made-project.git /home/cip/ce/ix05ogym/Majid/LLM/llm_made/data//0/
  stderr: 'fatal: destination path '/home/cip/ce/ix05ogym/Majid/LLM/llm_made/data//0' already exists and is not an empty directory.
'

In [15]:
repo_path_list = [data_path + f'{p}/' for p in os.listdir(data_path)]


In [31]:
from unstructured.partition.pdf import partition_pdf

# Define the path to your PDF file
pdf_file_path = "your_file.pdf"

# Extract content from the PDF

# Print the extracted text


def get_data(p):
        p += 'project/'
        code_path =  p + 'pipeline.py'
        data_report_path = p + 'data-report.pdf'
        analysis_report_path =  p + 'analysis-report.pdf'
        code = ''
        with open(code_path,'r') as file :
                code  = file.read()
                
        elements = partition_pdf(filename=data_report_path)
        data_report = ''
        for element in elements:
            data_report +=element.text
            
        elements = partition_pdf(filename=analysis_report_path)
        analysis_report = ''
        for element in elements:
           analysis_report +=element.text
            
        return code,data_report,analysis_report
            
prompt="""##
Pipline code:
{code}
##
Report:
{report}
##

You are given a code and report. You need to find some good datasets related to America's economy, society, .... and merge all the data based on some criteria. then write an ETL data pipeline for that. then analysis the created dataset and answer an important question.
"""
for p in repo_path_list:            
    code,data_report,analysis_report = get_data(p)  
    fp = prompt.format(code=code,report= data_report)
    print(fp)
    break
    
    

##
Pipline code:
import requests
from lxml import html
from zipfile import ZipFile
import pandas as pd
from io import BytesIO, StringIO
from tqdm import tqdm
import re
import os

from sqlalchemy import create_engine

def get_data_files(url: str) -> tuple[pd.DataFrame, list[bytes]]:
	zip_files = []
	r = requests.get(url)

	webpage = html.fromstring(r.content)
	links = webpage.xpath('//a/@href')
	for link in tqdm(links):

		if link.endswith('.txt'):
			meta_data = extract_meta_data_from_link(url, link)
			continue

		if not link.endswith('.zip'):
			continue

		response = requests.get(url + link)
		zip_files.append(response.content)

	return meta_data, zip_files



def extract_meta_data_from_link(url, link) -> pd.DataFrame:
	response = requests.get(url + link)
	lines = []
	for i, line in enumerate(StringIO(response.text)):
		split1 = line.split(maxsplit=6)
		if i == 0:
			split2 = split1[-1].split()
		elif i == 1:
			continue
		else:
			split2 = split1[6].strip()
			split2 = re.split('\s